In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
#Check the present working directory using pwd command

In [ ]:
#!kaggle datasets download -d aashita/nyt-comments

In [1]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import string, os
import pandas as pd
import numpy as np
from keras import utils as np_utils
from tqdm import tqdm

In [2]:
import pandas as pd
import string
songs = pd.read_csv('./labeled_lyrics_cleaned.csv')
lyrics = songs.seq
lyrics_cnt = len(lyrics)

all_lyrics = []
all_lyrics.extend(list(lyrics.values))
#print(all_lyrics)

for a in all_lyrics:
  if not isinstance(a, str):
    all_lyrics.remove(a)

print(len(all_lyrics))
# all_lyrics = all_lyrics[:200]

translator = str.maketrans('', '', string.punctuation)

all_lyrics = all_lyrics[0:1000]

# poetry = pd.read_csv('/content/gdrive/My Drive/Kaggle/PoetryFoundationData.csv',quotechar='"')

# poetry.head()

# poem = poetry.Poem
# poem_cnt = len(poem)

# all_poems = []
# all_poems.extend(list(poem.values))


# for a in all_poems:
#   if not isinstance(a, str):
#     all_poems.remove(a)

# all_poems = all_poems[:50]
# joined = all_poems+all_lyrics







158353


In [3]:

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for song in tqdm(corpus):
        prev_token_list = []
        for line in song.split('\r\n'):
            if not line:
                continue
#             print(line)
            token_list = list(tokenizer.texts_to_sequences([line])[0])
            #print(token_list)
            # input_sequences.append(list(prev_token_list) + list(token_list))
            two_lines = prev_token_list + token_list
            for i in range(1, len(two_lines)):
                n_gram_sequence = two_lines[:i+1]
                input_sequences.append(n_gram_sequence)
#             print(input_sequences)
            prev_token_list = token_list

    

    return input_sequences, total_words

def get_padded_sequences(input_sequences):
    max_seq_len = max( [len(x) for x in input_sequences])
    padded_input_sequences =  np.array(pad_sequences(input_sequences, 
                                                     maxlen=max_seq_len, padding='pre'))
    return padded_input_sequences, max_seq_len


def prep_train_data(padded_input_sequences,total_words):
    
    x_train = padded_input_sequences[:,:-1]
    labels = padded_input_sequences[:,-1]
    labels = keras.utils.np_utils.to_categorical(labels, num_classes=total_words)
    
    return x_train, labels

In [4]:
import re
rgx = re.compile(r"\[[^\n\]]*]")
all_lyrics = songs.seq.apply(lambda x: rgx.sub('', x))
all_lyrics = all_lyrics[0:200]

In [5]:
import enchant

d = enchant.Dict("en_US")

all_lyrics_en = []

for song in tqdm(all_lyrics):
    _cnt_y = 0
    _cnt_n = 0
    for line in song.split('\r\n'):
        for word in line.split():
            if d.check(word):
                _cnt_y += 1
            else:
                _cnt_n += 1
    if _cnt_n > _cnt_y:
        print(song)
        continue
    all_lyrics_en.append(song)
                
all_lyrics = all_lyrics_en

 10%|██████████████▋                                                                                                                                            | 19/200 [00:00<00:02, 89.83it/s]

Dieses ist lange her.
Da ich deine schmalen Hande nahm
Dieses ist lange her.
Schlugst die runden Augen auf, 
Dieses ist lange her.

Am Abend, 
Wenn wir auf dunklen Pfaden gehn, 
Erscheinen unsere bleichen Gestalten. 

Wenn uns durstet, 
Trinken wir die weissen Wasser des Teichs, 
Die Susse unserer traurigen Kindheit. 

Estorbene
Ruhen wir unterm Gebusch, 
Schaun den grauen Mowen zu. 

Fruhlingsgewolke 
Streigen uber die Stadt,
Die der Monche edlere Zeiten schweigt. 

Doch wenn dunkler Wohllaut 
Die Seele heimsucht, 
Erscheinst do Weisse in des Freundes Landschaft. 

Dieses ist lange her.
Da ich deine schmalen Hande nahm
Dieses ist lange her.
Schlugst die runden Augen auf, 
Dieses ist lange her.
Nebel zieh'n gespentisch vor 
Der Sucher setzt leis seinen Fuss 
Einsam unterm Sternenzelt 

Rastlose Suche in der Nacht 
Wandelnd zwischen Schlaf und Tod 
Nach einem Schattentraum

Furcht und Sehnsucht 
Folgen sinem Weg 
I'm Dammergarten scheint ein fahles Licht 

Engel der Nacht 
Ich warte auf

 20%|███████████████████████████████                                                                                                                            | 40/200 [00:00<00:01, 95.39it/s]

Deve ser bom, deve ser bom
To fechado pra balanco
Meu saldo deve ser bom
Deve  ser bom
Vou sambar de roda um pouco
Um xaxado bem guardado
E mais algum trocado
Se tiver gingado eu T, eu t, eu t
Eu t de corpo fechado, eu t, eu t
Deve ser bom, deve ser bom
T fechado pra balanco
Meu saldo deve ser bom
Um pouco da minha grana

Vais ter saudade, baiana
Ponho sempre por semana
Cinco cartas no correio
Gasto sola de sapato
Mas aqui custa barato
Cada sola de sapato
Custa um samba, um samba e meio
O resto
O resto nao d despesa
Viver nao me custa nada
Viver s me custa a vida
E a minha vida foi dada


 64%|██████████████████████████████████████████████████████████████████████████████████████████████████▌                                                       | 128/200 [00:01<00:00, 89.08it/s]

Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo 

One, two, three, four, five, six, seven, eight 
A, B, C, D, E, F, G 
Hoodoo, voodoo, seven, twenty-one, two 
Haystack, hoe-stack, hey do the hoe-ta 
High boga, low joker, ninety-nine, a zero 
Sidewalk, streetcar, dance a goofy dance 

Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo 

One, two, three, four, five, six, seven, eight 
A, B, C, D, E, F, G 
Blackbird, blue jay, one, two, three, four 
Trash-stack, jump back, E, F, G 
Big man, little man, fat man, skinny man 
Grasshopper, green snake, hold my hand

Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-choo, chooka-chooka, choo-choo 
Choo-choo, choo-c

 87%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                    | 174/200 [00:01<00:00, 103.41it/s]

Dari stesen ke stesen naluri bertanya
Masih jauh lagikah destinasi di mimpi
Mengapakah terjadi sehingga tiada lagi
Yang sanggup hulurkan sekadar informasi

Dari hati ke hati kita terus meragu
Kemungkinan jawapan tersirat dalam diri
Yang kerap ditabalkan oleh para pujangga
Satu dalam sejuta buih dalam lautan
Manusia serumpun dari asal yang satu

Yang surut mengapa akhir ke akhir zaman
Istilah lawan perang masih wujud dan gagah
Apakah ini takdir atau rekaan kita

Kami anak zaman ini kini menyahut tugas
Untuk mengibarkan bendera damai
Moga anak kami tak lagi bermimpi
Tu... Tu...Tu... Tu...
Tu... Tu...Tu... Tu...

Nuri, sungguh aku terharu
Bila, kau masih dendangkan lagu
Asyikmu, yang menghiburkan
Jiwa, diwaktu dirimu terjerat...

Apakah kau tak merindu
Pada awan dan bayu
Pada gurun dan rimba... di sana...

Balik, jejak sutera
Sekabut, syurga sekecil terati

Nuri sungguh aku terharu
Bila, kau masih dayukan laguku

Apakah kau kan merindu
Pada awan dan bayu
Pada gurun dan rimba... sahaja...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 89.94it/s]


In [6]:
# corpus = [clean_text(x) for x in all_lyrics]
corpus = all_lyrics
tokenizer = Tokenizer()
input_sequences, total_words = get_sequence_of_tokens(corpus)
padded_input_sequences, max_seq_len = get_padded_sequences(input_sequences)
x_train, labels = prep_train_data(padded_input_sequences,total_words)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 184/184 [00:00<00:00, 4654.94it/s]


In [7]:
def lstm_model(max_seq_len,total_words):
    
    input_len = max_seq_len - 1 #zadnju predvidam
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(128))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = lstm_model(max_seq_len,total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 418, 10)           37040     
                                                                 
 lstm (LSTM)                 (None, 128)               71168     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 3704)              477816    
                                                                 
Total params: 586,024
Trainable params: 586,024
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(x_train, labels, epochs=60, verbose=1)

Epoch 1/60
1957/1957 [==============================] - 41s 20ms/step - loss: 6.0593
Epoch 2/60
1957/1957 [==============================] - 39s 20ms/step - loss: 5.5691
Epoch 3/60
1957/1957 [==============================] - 39s 20ms/step - loss: 5.2152
Epoch 4/60
1957/1957 [==============================] - 39s 20ms/step - loss: 4.8286
Epoch 5/60
1957/1957 [==============================] - 39s 20ms/step - loss: 4.6080
Epoch 6/60
1957/1957 [==============================] - 39s 20ms/step - loss: 4.3034
Epoch 7/60
1957/1957 [==============================] - 39s 20ms/step - loss: 4.0346
Epoch 8/60
1957/1957 [==============================] - 39s 20ms/step - loss: 3.8001
Epoch 9/60
1957/1957 [==============================] - 39s 20ms/step - loss: 3.5842
Epoch 10/60
1957/1957 [==============================] - 38s 20ms/step - loss: 3.3943
Epoch 11/60
1957/1957 [==============================] - 39s 20ms/step - loss: 3.2288
Epoch 12/60
1957/1957 [==============================] - 39s 20

In [9]:
def generate_lyrics(seed_txt, next_words_cnt , max_seq_len, model):
  
    for i in range(0, next_words_cnt):
        token_list = tokenizer.texts_to_sequences([seed_txt])[0]
        padded_token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        prediction = np.argmax(model.predict(padded_token_list), axis=-1)

        #print('prediction',prediction)
        #tokenizer.word_index.items() --> rijecnik (rijec,index)
        
        
        for (word, index) in tokenizer.word_index.items():
            output_word = ""
            if (prediction == index):
                seed_txt += " " + word
                break
      
        
    return seed_txt.title()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


In [ ]:
#model.save("model_franzi")

In [10]:
cnt = 0
_cnt = 0
for l in all_lyrics:
    for _l in l.split('\r\n'):
        if len(_l.split()) == 1:
            cnt += 1
        _cnt += 1
            
print(cnt)
print(_cnt)

85
5893


In [11]:
import random
unique = []
for i in range(0,len(corpus)):
    unique.append(corpus[i].split())


for i in range(0,20):
  random_list = unique[random.randint(0,len(unique)-1)]
  seed = random_list[random.randint(0,len(random_list))]
  print( generate_lyrics(seed,random.randint(3,15),max_seq_len, model ) )








Me Dio Dos Luceros Que Cuando Los Abro Perfecto Distingo Lo Negro
That, You Just A Little Bird Told Me So I Go To Sleep
Benz Say I Love You Although We Are All The Meaning To The Moon
Look At The Ashes You Came Down The Lights Eyes
A Shower Of Pure Joy Love Is All All We Want A Shower Of
Don'T You Be A Dream Until We'Ll Stay Best Of The Rest Of
Street And I Know Eventually We'Ll Be Together One Sweet Day If You Think
Of You I Need To Be My Friend And Let You Wrap You In My Warm
Beautiful World Circling Fragment Of Sun Marbled In Blue Turning And Bright Man
Darling My Darling Will Shine In You
Will Wash Away Our Pain Never Doubt Never Be Waiting For You 'Cause It
I Know I Can Do I Can'T Stand Down To You Found
Bar Me And I
Sky My Confession It'S My Fault I Could Ever Never Be Love 'Cause Love It Was
Little Bird Little Bird
Some Morning Rain Will Wash Away Our
They Say Let'S Die Crazy And Is An Last
Around And Close To Be London Inside And I Won'T You Baby Baby Baby Baby
California Dream